# Ray RLlib Multi-Armed Bandits - A Simple Bandit Example

© 2019-2020, Anyscale. All Rights Reserved

![Anyscale Academy](../../images/AnyscaleAcademy_Logo_clearbanner_141x100.png)

Let's explore a very simple contextual bandit example with three arms. We'll run trials using RLlib and [Tune](http://tune.io), Ray's hyperparameter tuning library. 

In [1]:
import gym
from gym.spaces import Discrete, Box
import numpy as np
import random, time
import ray

We define the bandit as a subclass of an OpenAI Gym environment. We set the action space to have three discrete variables, one action for each arm, and an observation space (the context) in the range -1.0 to 1.0, inclusive. 

There are two contexts defined. Note that we'll randomly pick one of them to use when `reset` is called, but it stays fixed (static) throughout the episode (the set of steps between calls to `reset`).

In [2]:
class SimpleContextualBandit (gym.Env):
    def __init__ (self, config=None):
        self.action_space = Discrete(3)     # 3 arms
        self.observation_space = Box(low=-1., high=1., shape=(2, ), dtype=np.float64)  # Random (x,y), where x,y from -1 to 1
        self.current_context = None
        self.rewards_for_context = {
            -1.: [-10, 0, 10],
            1.: [10, 0, -10],
        }

    def reset (self):
        self.current_context = random.choice([-1., 1.])
        return np.array([-self.current_context, self.current_context])

    def step (self, action):
        reward = self.rewards_for_context[self.current_context][action]
        return (np.array([-self.current_context, self.current_context]), reward, True,
                {
                    "regret": 10 - reward
                })

    def __repr__(self):
        return f'SimpleContextualBandit(action_space={self.action_space}, observation_space={self.observation_space}, current_context={self.current_context}, rewards per context={self.rewards_for_context})'

Look at the definition of `self.rewards_for_context`. For context `-1.`, choosing the **third** arm (index 2 in the array) maximizes the reward, yielding `10.0` for each pull. Similarly, for context `1.`, choosing the **first** arm (index 0 in the array) maximizes the reward. It is never advantageous to choose the second arm.

We'll see if our training results agree ;)

Try repeating the next two code cells enough times to see the `current_context` set to `1.0` and `-1.0`, which is done randomly by `reset`.

In [3]:
bandit = SimpleContextualBandit()
observation = bandit.reset()
f'Initial observation = {observation}, bandit = {repr(bandit)}'

'Initial observation = [-1.  1.], bandit = SimpleContextualBandit(action_space=Discrete(3), observation_space=Box(2,), current_context=1.0, rewards per context={-1.0: [-10, 0, 10], 1.0: [10, 0, -10]})'

The `bandit.current_context` and the observation of the current environment will remain fixed through the episode.

In [4]:
print(f'current_context = {bandit.current_context}')
for i in range(10):
    action = bandit.action_space.sample()
    observation, reward, done, info = bandit.step(action)
    print(f'observation = {observation}, action = {action}, reward = {reward:4d}, done = {str(done):5s}, info = {info}')

current_context = 1.0
observation = [-1.  1.], action = 2, reward =  -10, done = True , info = {'regret': 20}
observation = [-1.  1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [-1.  1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [-1.  1.], action = 2, reward =  -10, done = True , info = {'regret': 20}
observation = [-1.  1.], action = 0, reward =   10, done = True , info = {'regret': 0}
observation = [-1.  1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [-1.  1.], action = 2, reward =  -10, done = True , info = {'regret': 20}
observation = [-1.  1.], action = 1, reward =    0, done = True , info = {'regret': 10}
observation = [-1.  1.], action = 2, reward =  -10, done = True , info = {'regret': 20}
observation = [-1.  1.], action = 0, reward =   10, done = True , info = {'regret': 0}


Look at the `current_context`. If it's `1.0`, does the `0` (first) action yield the highest reward and lowest regret? If it's `-1.0`, does the `2` (third) action yield the highest reward and lowest regret?

## Using LinUCB

For this simple example, we can easily determine the best actions to take. Let's see how well our system does. We'll train with [LinUCB](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-upper-confidence-bound-contrib-linucb), a linear version of _Upper Confidence Bound_, for the exploration-exploitation strategy. _LinUCB_ assumes a linear dependency between the expected reward of an action and its context. Recall that a linear function is of the form $z = ax + by + c$, for example, where $x$, $y$, and $z$ are variables and $a$, $b$, and $c$ are constants. _LinUCB_ models the representation space using a set of linear predictors. Hence, the $Q_t(a)$ function discussed for UCB in the [previous lesson](02-Exploration-vs-Exploitation-Strategies.ipynb) is a linear function here.

Look again at how we defined `rewards_for_context`. Is it linear as expected for _LinUCB_?

```python
self.rewards_for_context = {
    -1.: [-10, 0, 10],
    1.: [10, 0, -10],
}
```

Yes, for each arm, the reward is linear in the context. For example, the first arm has a reward of `-10` for context `-1.0` and `10` for context `1.0`. Crucially, the _same_ linear function that works for the first arm will work for the other two arms if you multiplied the constants in the linear function by `0` and `-1`, respectively. Hence, we expect _LinUCB_ to work well for this example.

Now use Tune to train the policy for this bandit. But first, we want to make sure that Ray is running and we want to initialize Ray in this "driver" program explicitly. Note that the call to `ray.tune.run` in a few cells below would handle initialization for us, if we omitted the `ray_auto_init=False`, but that can lead to multiple Ray clusters running, so we'll control it more tightly. Hence, we now initialize Ray, then setup our experiment.

The following shell command will probably print _INFO: Ray is already running._ If not, follow the instructions it prints to start Ray.

In [5]:
!../../tools/start-ray.sh --check --verbose

INFO: Ray is already running.


Now initialize Ray in this "driver" program (notebook).

In [6]:
ray.init(address='auto', ignore_reinit_error=True)

2020-06-13 09:25:57,558	WARNING worker.py:809 -- When connecting to an existing cluster, _internal_config must match the cluster's _internal_config.


{'node_ip_address': '192.168.1.149',
 'raylet_ip_address': '192.168.1.149',
 'redis_address': '192.168.1.149:15832',
 'object_store_address': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-06-12_08-58-38_626987_40764'}

In [7]:
stop = {
    "training_iteration": 200,
    "timesteps_total": 100000,
    "episode_reward_mean": 10.0,
}

config = {
    "env": SimpleContextualBandit,
}

In [8]:
from ray.tune.progress_reporter import JupyterNotebookReporter

In [9]:
start_time = time.time()

analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
                        progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
                        verbose=2,     # Change to 0 or 1 to reduce the output.
                        ray_auto_init=False,    # Don't allow Tune to initialize Ray.
                       )

Trial name,status,loc
contrib_LinUCB_SimpleContextualBandit_00000,RUNNING,


(pid=74572) 2020-06-13 09:26:26,394	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=74572) 2020-06-13 09:26:26,398	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74572) 2020-06-13 09:26:26,409	INFO trainable.py:217 -- Getting current IP.
(pid=74572) 2020-06-13 09:26:26,409	WARNING util.py:37 -- Install gputil for GPU system monitoring.
Result for contrib_LinUCB_SimpleContextualBandit_00000:
  custom_metrics: {}
  date: 2020-06-13_09-26-26
  done: false
  episode_len_mean: 1.0
  episode_reward_max: 10.0
  episode_reward_mean: 9.9
  episode_reward_min: 0.0
  episodes_this_iter: 100
  episodes_total: 100
  experiment_id: e5b14c156ae34b5f99203db213772505
  experiment_tag: '0'
  grad_time_ms: 0.258
  hostname: DWAnyscaleMBP.local
  info:
    grad_time_ms: 0.258
    learner:
      cumulative_regret: 10.0
      update_latency: 0.0001411437988

Trial name,status,loc,iter,total time (s),ts,reward
contrib_LinUCB_SimpleContextualBandit_00000,TERMINATED,,2,0.510471,200,10


In [10]:
print("The trials took", time.time() - start_time, "seconds\n")

The trials took 24.94225788116455 seconds



We can see some of the final data as a dataframe:

In [11]:
df = analysis.dataframe()
df

,episode_reward_max,episode_reward_min,episode_reward_mean,episode_len_mean,episodes_this_iter,num_steps_trained,num_steps_sampled,sample_time_ms,grad_time_ms,update_time_ms,...,info/sample_peak_throughput,info/opt_samples,learner/cumulative_regret,learner/update_latency,perf/cpu_util_percent,perf/ram_util_percent,info/learner/cumulative_regret,info/learner/update_latency,config/env,logdir
0,10.0,10.0,10.0,1.0,100,200,200,2.93,1.066,0.01,...,341.353,1.0,10.0,0.000134,NaN,NaN,10.0,0.000134,<class '__main__.SimpleContextualBandit'>,/Users/deanwampler/ray_results/contrib/LinUCB/...


(pid=74570) 2020-06-13 09:29:54,612	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=74570) 2020-06-13 09:29:54,616	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74570) 2020-06-13 09:29:54,625	INFO trainable.py:217 -- Getting current IP.
(pid=74570) 2020-06-13 09:29:54,626	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=74571) 2020-06-13 09:30:50,690	INFO trainer.py:421 -- Tip: set 'eager': true or the --eager flag to enable TensorFlow eager execution
(pid=74571) 2020-06-13 09:30:50,696	INFO trainer.py:580 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=74571) 2020-06-13 09:30:50,707	INFO trainable.py:217 -- Getting current IP.
(pid=74571) 2020-06-13 09:30:50,708	WARNING util.py:37 -- Install gputil for GPU system monitoring.
(pid=75143) 2020-0

The easiest way to inspect the progression of training is to use TensorBoard.

1. If you are runnng on the Anyscale Platform, click the _TensorBoard_ link. 
2. If you running this notebook on a laptop, open a terminal window using the `+` under the _Edit_ menu, run the following command, then open the URL shown.

```
tensorboard --logdir ~/ray_results 
```

You may have many data sets plotted from previous tutorial lessons. In the _Runs_ on the left, look for one named something like this:

```
contrib/LinUCB/contrib_LinUCB_SimpleContextualBandit_0_YYYY-MM-DD_HH-MM-SSxxxxxxxx  
```

If you have several of them, you want the one with the latest timestamp. To select just that one, click _toggler all runs_ below the list of runs, then select the one you want. You should see something like [this image](../../images/rllib/TensorBoard1.png).

The graph for the metric we were optimizing, the mean reward, is shown with a rectangle surrounding it. It improved steadily during the training runs. For this simple example, the reward mean is easily found in 200 steps.

## Exercise 1

Change the the `step` method to randomly change the `current_context` on each invocation:

```python
def step(self, action):
    result = super().step(action)
    self.current_context = random.choice([-1.,1.])
    return (np.array([-self.current_context, self.current_context]), reward, True,
            {
                "regret": 10 - reward
            })
```

Repeat the training and analysis. Does the training behavior change in any appreciable way? Why or why not?

See the [solutions notebook](solutions/Multi-Armed-Bandits-Solutions.ipynb) for discussion of this and the following exercises.

## Exercise 2

Recall the `rewards_for_context` we used:

```python
self.rewards_for_context = {
    -1.: [-10, 0, 10],
    1.: [10, 0, -10],
}
```

We said that Linear Upper Confidence Bound assumes a linear dependency between the expected reward of an action and its context. It models the representation space using a set of linear predictors.

Change the values for the rewards as follows, so they no longer have the same simple linear relationship:

```python
self.rewards_for_context = {
    -1.: [-10, 10, 0],
    1.: [0, 10, -10],
}
```

Run the training again and look at the results for the reward mean in TensorBoard. How successful was the training? How smooth is the plot for `episode_reward_mean`? How many steps were taken in the training?

## Exercise 3 (Optional)

We briefly discussed another algorithm for selecting the next action, _Thompson Sampling_, in the [previous lesson](02-Exploration-vs-Exploitation-Strategies.ipynb). Repeat exercises 1 and 2 using linear version, called _Linear Thompson Sampling_ ([RLlib documentation](https://docs.ray.io/en/latest/rllib-algorithms.html?highlight=greedy#linear-thompson-sampling-contrib-lints)). To make this change, look at this code we used above:

```python
analysis = ray.tune.run("contrib/LinUCB", config=config, stop=stop, 
                        progress_reporter=JupyterNotebookReporter(overwrite=False),  # This is the default, actually.
                        verbose=2,              # Change to 0 or 1 to reduce the output.
                        ray_auto_init=False)    # Don't allow Tune to initialize Ray.
```

Change `contrib/LinUCB` to `contrib/LinTS`.  

We'll continue exploring usage of _LinUCB_ in the next lesson, [04 Linear Upper Confidence Bound](04-Linear-Upper-Confidence-Bound.ipynb.ipynb) and _LinTS_ in the following lesson, [05 Thompson Sampling](05-Thompson-Sampling.ipynb).